In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from time import clock

In [2]:
begin_page = 1
end_page = 2
master_page = 'https://www.redkaraoke.com/karaoke/celine-dion/my-heart-will-go-on/11827/recordings/p'
basePlayerUrl = "https://www.redkaraoke.com";
baseS3Url = "https://s3.amazonaws.com/";
#js file: https://d4rgvppl55fks.cloudfront.net/es/js/playerRecs.js

In [3]:
links_list = []
for i in range(begin_page, end_page, 1):
    r = requests.get(master_page + str(i))
    soup = BeautifulSoup(r.content, 'html.parser')
    for link in soup.find_all('a', attrs = {'class':'result result-recording'}):
        recording_link = 'https://www.redkaraoke.com' + link.get('href')
        links_list.append(recording_link)
#         print(recording_link)
        
# Remove duplicated links
links_list_unique = set(links_list)

In [4]:
print(len(links_list))
print(len(links_list_unique))

24
24


In [5]:
def extractUserRecording(soup):
# Localize data in "userRecording" object
    pattern = re.compile(r"userRecording")
    data = soup.find('script', text=pattern)

    pattern = re.compile(r"userRecording\.([a-zA-Z0-9]+)=([^;]+);")
    data = soup.find('script', text=pattern)
    ur_string = re.findall(r"userRecording\.([a-zA-Z0-9]+)=([^;]+);",str(data))
    ur_dict = dict(ur_string)
    for key in ur_dict.keys():
        ur_dict[key] = ur_dict[key][1:-1]
    return ur_dict 

In [6]:
def getDownloadLink(ur_dict):
    isDueto = False

    if 'isDueto' in ur_dict.keys(): 
        isDueto = True
    else:
        isDueto = False

    if ur_dict['file'][-1] == '3':
        file_type = 'mp3'
        if ur_dict['lang'] == 'es':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/mp3/es/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraoke/data/mp3/"
            else:
                filepath = basePlayerUrl + "/data/mp3/es/"
        elif ur_dict['lang'] == 'jp':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/mp3/jp/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokejp/data/mp3/"
            else:
                filepath = baseS3Url + "/data/mp3/jp/"
        else:
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/mp3/en/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokecom/data/mp3/"
            else:
                filepath = basePlayerUrl + "/data/mp3/en/"
    else:
        file_type = 'vid'
        if ur_dict['lang'] == 'es':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/vid/es/"
            elif 'isDueto' in ur_dict.keys():
                filepath = baseS3Url + "redkaraoke/data/vid/"
            else:
                filepath = basePlayerUrl + "/data/vid/es/"
        elif ur_dict['lang'] == 'jp':
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/vid/jp/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokejp/data/vid/"
            else:
                filepath = baseS3Url + "/data/vid/jp/"
        else:
            if isDueto:
                filepath = basePlayerUrl + "/data/duets/vid/en/"
            elif 's3' in ur_dict.keys():
                filepath = baseS3Url + "redkaraokecom/data/vid/"
            else:
                filepath = basePlayerUrl + "/data/vid/en/"
    download_link = filepath + ur_dict['file']
    return download_link, file_type, isDueto

In [7]:
counter = 0

#Initialize lists
view_list = []
like_list = []
comment_list = []
recording_list = []
follower_list = []
following_list = []
date_list = []

location_list = []
user_avatar_list = []
user_group_list = []
user_name_list = []
language_list = []
song_title_list = []
artist_list = []

download_link_list = []
recording_link_list = []
file_type_list = []
is_duet_list = []
gender_list = []
country_list = []
recording_id_list = []

# Send request for each link in links_list_unique
for link_sub in links_list_unique:
    r = requests.get(link_sub)
    soup = BeautifulSoup(r.content, 'lxml')
    
    try:
        counter = counter + 1
        
        # Extract information from html code:
        view = int(soup.find('li', attrs = {'class':'rec-views'}).text)
        like = soup.find('li', attrs = {'class':'rec-likes'}).text
        comment = int(soup.find('li', attrs = {'class':'rec-comments'}).text)
        recording = int(soup.find('li', attrs = {'class':'tam1'}).find('strong').text)
        follower = int(soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[5].find('strong').text)
        following = (soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[6].find('strong').text)
        date = soup.find_all('div', attrs = {'class':'infoTema_textoBlanco'})[2].text[3:]
        gender_loc = soup.find('div', attrs = {'class':'userdata'}).find('p').text
        
        if gender_loc.split(',')[0] == 'Man':
            gender = 'man'
        elif gender_loc.split(',')[0] == 'Woman':
            gender = 'woman'
        else:
            gender = float('nan')
        country = gender_loc.split()[-1]
            
        recording_link = link_sub

        ur_dict = extractUserRecording(soup)
        download_link, file_type, is_duet = getDownloadLink(ur_dict)
        print(download_link)
        
        location = ur_dict['loc']
        user_avatar = ur_dict['uavatar']
        user_group = ur_dict['ugroup']
        user_name = ur_dict['u']
        language = ur_dict['lang']
        song_title = ur_dict['title']
        artist = ur_dict['a']
        recording_id = ur_dict['rid']
                
    except (IndexError, AttributeError) as e:
        continue
        
    view_list.append(view)
    like_list.append(like)
    comment_list.append(comment)
    recording_list.append(recording)
    follower_list.append(follower)
    following_list.append(following)
    date_list.append(date)    
    gender_list.append(gender)
    country_list.append(country)
    
    location_list.append(location)
    user_avatar_list.append(user_avatar)
    user_group_list.append(user_group)
    user_name_list.append(user_name)
    language_list.append(language)
    song_title_list.append(song_title)
    artist_list.append(artist)
    download_link_list.append(download_link)
    recording_link_list.append(recording_link)
    file_type_list.append(file_type)
    is_duet_list.append(is_duet)
    recording_id_list.append(recording_id)
    print(counter)

https://s3.amazonaws.com/redkaraoke/data/mp3/11827/cf8281d22d136c250038c34d424da24a_1.mp3
1
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/05d2bc50fa9e0ea8a4759bfc0c1be19a_1.mp3
2
https://www.redkaraoke.com/data/mp3/es/11827/34fd64bd49ea989deb89a08b0989d4d7_1.mp3
3
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/9fe9ac9350ded7caa9e66f38fdb4187d_1.mp4
4
https://s3.amazonaws.com/redkaraokejp/data/vid/11827/81352dfcf78542cd24af7d8250c73a5a_1.mp4
5
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/5e998bf3eb925b12d70277c972fc030d_1.mp3
6
https://www.redkaraoke.com/data/mp3/es/11827/63735e1c969fd219c4b0d30b5ce592aa_1.mp3
7
https://www.redkaraoke.com/data/mp3/es/11827/50368f63f9773b104232f40a993471a1_1.mp3
8
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/cc3a10d9e61120e35771413fc05b5209_1.mp4
9
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/2ea24cee588c4b0d7b30b0e1d6b364c6_1.mp3
10
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/0ddb0460d6e687f02fbc506d809fa0a7_1.

In [8]:
print(len(view_list))

22


In [9]:
d = {'no_of_views': view_list, 'no_of_likes': like_list, 'no_of_comments': comment_list, 
     'no_of_recordings': recording_list, 'no_of_followers': follower_list, 'no_of_following': following_list,
     'date_recorded': date_list, 'gender': gender_list, 'country': country_list,
     'location': location_list, 'user_avatar': user_avatar_list, 'user_group': user_group_list,
     'user_name': user_name_list, 'language': language_list, 'song_title': song_title_list, 
     'artist': artist_list, 'download_link': download_link_list, 'recording_link': recording_link_list,
     'file_type': file_type_list, 'is_duet': is_duet_list, 'recording_id': recording_id_list}

In [10]:
df = pd.DataFrame(data=d)

In [11]:
df['date_recorded'] = pd.to_datetime(df['date_recorded'])
df['today'] = pd.Timestamp('20171023')
df['days'] = (df['today'] - df['date_recorded']).dt.days
df = df.drop('today',axis=1)

In [12]:
df.head()

,artist,country,date_recorded,download_link,file_type,gender,is_duet,language,location,no_of_comments,...,no_of_likes,no_of_recordings,no_of_views,recording_id,recording_link,song_title,user_avatar,user_group,user_name,days
0,Celine Dion,Uruguay,2010-04-29,https://s3.amazonaws.com/redkaraoke/data/mp3/1...,mp3,woman,False,es,en,29,...,24,501,251,722120,https://www.redkaraoke.com/profile/tinarosy/re...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,tinarosy,2734
1,Celine Dion,Spain,2010-02-08,https://s3.amazonaws.com/redkaraoke/data/mp3/1...,mp3,woman,False,es,en,29,...,29,98,593,648309,https://www.redkaraoke.com/profile/Arena55/rec...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,Arena55,2814
2,Celine Dion,Argentina,2009-04-29,https://www.redkaraoke.com/data/mp3/es/11827/3...,mp3,woman,False,es,en,28,...,22,58,283,448053,https://www.redkaraoke.com/profile/adr1ana/rec...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,adr1ana,3099
3,Celine Dion,Japan,2016-01-28,https://s3.amazonaws.com/redkaraokecom/data/vi...,vid,woman,False,en,en,71,...,82,462,1492,565862,https://www.redkaraoke.com/profile/fruitynao/r...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,2,fruitynao,634
4,Celine Dion,Japan,2015-11-12,https://s3.amazonaws.com/redkaraokejp/data/vid...,vid,man,False,jp,en,42,...,32,862,1869,297890,https://www.redkaraoke.com/profile/yasumiyo/re...,My heart will go on,https://d4rgvppl55fks.cloudfront.net/forum/cus...,9,yasumiyo,711


In [13]:
df.to_csv('./data/out' + '_' + str(begin_page) + '_' + str(end_page) + '.csv',index=False)